In [2]:
from dataset import get_datasets

train_dataset, test_dataset = get_datasets()

In [3]:
import pandas as pd
import torch.nn as nn
from torchvision import models
from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
resnet18 = models.resnet18(pretrained=True)
last_layer_classifier = nn.Sequential(
    nn.Linear(resnet18.fc.in_features, len(df.columns) - 2), 
    nn.Sigmoid()
)

/Users/yslomichael/opt/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yslomichael/opt/anaconda3/envs/stat4012proj/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
import wandb

PROJECT_NAME = "STAT4012Project"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from torch import optim
from model import pretrainedModel

config = {
    "batch_size": 32,
    "loss_fn": nn.BCELoss,
    "lr": 5e-5,
    "optimizer": optim.Adam,
    "epochs": 5,
}

model_resnet18 = pretrainedModel(
    resnet18,
    last_layer_classifier,
    len(df.columns) - 2,
    config["loss_fn"],
    config["optimizer"],
    lr=config["lr"],
)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True,num_workers=6, pin_memory=True)
validation_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False,num_workers=6, pin_memory=True)

run_name = "resnet18_" + str(int(time.time()))
logger = WandbLogger(name=run_name, project=PROJECT_NAME, log_model="all")
trainer = L.Trainer(max_epochs=config["epochs"], logger=logger, accelerator="gpu", devices=1, default_root_dir="C:\Users\USER\Desktop\STAT4012\ResNet18")

trainer.fit(model_resnet18, train_loader, validation_loader)